# RAG System Testing Notebook

This notebook allows you to test and verify the RAG (Retrieval-Augmented Generation) system for both Clarity and Rigor agents.

## Prerequisites

1. Qdrant running: `docker run -p 6333:6333 qdrant/qdrant`
2. Documents embedded: `python scripts/embed_documents.py --all`
3. Environment variables set (`.env` file with `OPENAI_API_KEY`)

## Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path().absolute().parent))

import time
from qdrant_client import QdrantClient
from app.rag.caching import create_cached_embeddings
from app.rag.config import clarity_rag_config, rigor_rag_config
from app.rag.rag_service import RAGService
from app.agents.rag_nodes import ClarityRAGNode, RigorRAGNode
from app.models.schemas import Section

print("✓ Imports successful")

## 1. Check Qdrant Connection and Collections

In [ ]:
# Connect to Qdrant
qdrant_client = QdrantClient(url="http://localhost:6333")

try:
    # Get all collections
    collections = qdrant_client.get_collections()
    
    print("\n" + "="*60)
    print("QDRANT COLLECTIONS")
    print("="*60)
    
    if not collections.collections:
        print("⚠️  No collections found!")
        print("   Run: python scripts/embed_documents.py --all")
    else:
        for collection in collections.collections:
            info = qdrant_client.get_collection(collection.name)
            print(f"\n📚 Collection: {collection.name}")
            print(f"   Points: {info.points_count}")
            print(f"   Vectors: {info.config.params.vectors}")
            
    print("\n✓ Qdrant connection successful")
    
except Exception as e:
    print(f"\n❌ Error connecting to Qdrant: {e}")
    print("   Make sure Qdrant is running: docker run -p 6333:6333 qdrant/qdrant")

## 2. Test Embedding Cache

Verify that caching is working properly.

In [ ]:
print("\n" + "="*60)
print("TESTING EMBEDDING CACHE")
print("="*60)

# Create cached embeddings
embeddings = create_cached_embeddings(namespace="test")

# Test texts
test_texts = [
    "Mathematical proofs require rigorous validation.",
    "Clear writing is essential for mathematical papers.",
    "Theorems should be stated precisely."
] * 10  # 30 texts total

# First run (no cache)
print(f"\n🔄 First run (embedding {len(test_texts)} texts)...")
start = time.time()
result1 = embeddings.embed_documents(test_texts)
first_time = time.time() - start
print(f"   Time: {first_time:.3f}s")
print(f"   Embeddings shape: {len(result1)} vectors of dimension {len(result1[0])}")

# Second run (cache hit)
print(f"\n⚡ Second run (using cache)...")
start = time.time()
result2 = embeddings.embed_documents(test_texts)
cache_time = time.time() - start
print(f"   Time: {cache_time:.3f}s")
print(f"   Speedup: {first_time/cache_time:.1f}x faster")

# Verify results are identical
assert result1 == result2, "Cache results should match!"
print("\n✓ Cache working correctly!")

# Show cache location
cache_dir = Path("app/embedding_cache/test")
if cache_dir.exists():
    cache_files = list(cache_dir.glob("*"))
    print(f"\n📂 Cache location: {cache_dir}")
    print(f"   Files: {len(cache_files)}")

## 3. Test Clarity Agent RAG

In [ ]:
print("\n" + "="*60)
print("TESTING CLARITY AGENT RAG")
print("="*60)

# Create RAG service for clarity agent
clarity_service = RAGService(config=clarity_rag_config, qdrant_client=qdrant_client)

# Test queries for clarity
clarity_queries = [
    "how to write clear mathematical statements",
    "avoiding complex sentences in technical writing",
    "defining jargon and technical terms",
    "improving readability of proofs"
]

for i, query in enumerate(clarity_queries, 1):
    print(f"\n{'─'*60}")
    print(f"Query {i}: {query}")
    print(f"{'─'*60}")
    
    try:
        # Retrieve documents
        start = time.time()
        results = clarity_service.retrieve(query, top_k=3)
        latency = time.time() - start
        
        print(f"⏱️  Latency: {latency*1000:.0f}ms")
        print(f"📄 Retrieved {len(results)} documents\n")
        
        # Display results
        for j, doc in enumerate(results, 1):
            source = doc.metadata.get('source', 'Unknown')
            agent_type = doc.metadata.get('agent_type', 'Unknown')
            
            print(f"{j}. Source: {source} (Agent: {agent_type})")
            print(f"   Preview: {doc.page_content[:200]}...")
            print()
            
    except Exception as e:
        print(f"❌ Error: {e}")
        print("   Make sure documents are embedded for clarity agent")

print("\n✓ Clarity RAG testing complete")

## 4. Test Rigor Agent RAG

In [ ]:
print("\n" + "="*60)
print("TESTING RIGOR AGENT RAG")
print("="*60)

# Create RAG service for rigor agent
rigor_service = RAGService(config=rigor_rag_config, qdrant_client=qdrant_client)

# Test queries for rigor
rigor_queries = [
    "mathematical proof validation and structure",
    "stating assumptions in theorems",
    "experimental design and statistical rigor",
    "checking mathematical correctness"
]

for i, query in enumerate(rigor_queries, 1):
    print(f"\n{'─'*60}")
    print(f"Query {i}: {query}")
    print(f"{'─'*60}")
    
    try:
        # Retrieve documents
        start = time.time()
        results = rigor_service.retrieve(query, top_k=3)
        latency = time.time() - start
        
        print(f"⏱️  Latency: {latency*1000:.0f}ms")
        print(f"📄 Retrieved {len(results)} documents\n")
        
        # Display results
        for j, doc in enumerate(results, 1):
            source = doc.metadata.get('source', 'Unknown')
            agent_type = doc.metadata.get('agent_type', 'Unknown')
            
            print(f"{j}. Source: {source} (Agent: {agent_type})")
            print(f"   Preview: {doc.page_content[:200]}...")
            print()
            
    except Exception as e:
        print(f"❌ Error: {e}")
        print("   Make sure documents are embedded for rigor agent")

print("\n✓ Rigor RAG testing complete")

## 5. Test RAG Nodes (Workflow Integration)

Test how RAG nodes work within the actual workflow.

In [ ]:
print("\n" + "="*60)
print("TESTING RAG NODES (WORKFLOW INTEGRATION)")
print("="*60)

# Create RAG nodes
clarity_rag_node = ClarityRAGNode(qdrant_client=qdrant_client)
rigor_rag_node = RigorRAGNode(qdrant_client=qdrant_client)

# Create sample sections
sample_sections = [
    Section(
        title="Introduction",
        content="We present a novel algorithm for optimization. The approach leverages sophisticated mathematical techniques to achieve optimal performance in various scenarios.",
        section_type="introduction",
        line_start=1
    ),
    Section(
        title="Methodology",
        content="Our proof relies on the following theorem: For all x in the domain, the function f(x) converges. We assume the conditions hold without explicit validation.",
        section_type="methodology",
        line_start=50
    )
]

# Test Clarity RAG Node
print("\n📘 Testing Clarity RAG Node")
print("─" * 60)

clarity_state = {
    "sections_for_clarity": sample_sections
}

clarity_result = clarity_rag_node(clarity_state)
clarity_guidelines = clarity_result.get("clarity_guidelines", "")

if clarity_guidelines:
    print(f"✓ Retrieved clarity guidelines ({len(clarity_guidelines)} chars)")
    print(f"\nGuidelines Preview:\n{clarity_guidelines[:500]}...")
else:
    print("⚠️  No clarity guidelines retrieved")

# Test Rigor RAG Node
print("\n\n📗 Testing Rigor RAG Node")
print("─" * 60)

rigor_state = {
    "sections_for_rigor": [sample_sections[1]]  # Only methodology section
}

rigor_result = rigor_rag_node(rigor_state)
rigor_guidelines = rigor_result.get("rigor_guidelines", "")

if rigor_guidelines:
    print(f"✓ Retrieved rigor guidelines ({len(rigor_guidelines)} chars)")
    print(f"\nGuidelines Preview:\n{rigor_guidelines[:500]}...")
else:
    print("⚠️  No rigor guidelines retrieved")

print("\n✓ RAG nodes testing complete")

## 6. Compare Retrieval Strategies

Compare naive vs other retrieval strategies (if available).

In [ ]:
print("\n" + "="*60)
print("COMPARING RETRIEVAL STRATEGIES")
print("="*60)

test_query = "how to improve clarity in mathematical writing"

# Test with naive retrieval
print(f"\nQuery: {test_query}")
print("\n1️⃣  Naive Retrieval")
print("─" * 60)

clarity_rag_config.retrieval_config.retriever_type = "naive"
naive_service = RAGService(config=clarity_rag_config, qdrant_client=qdrant_client)

start = time.time()
naive_results = naive_service.retrieve(test_query, top_k=3)
naive_latency = time.time() - start

print(f"⏱️  Latency: {naive_latency*1000:.0f}ms")
print(f"📄 Results: {len(naive_results)}")
for i, doc in enumerate(naive_results, 1):
    print(f"   {i}. {doc.metadata.get('source', 'Unknown')[:50]}")

# Try rerank if Cohere API key available
print("\n2️⃣  Rerank Retrieval (Cohere)")
print("─" * 60)

try:
    clarity_rag_config.retrieval_config.retriever_type = "rerank"
    rerank_service = RAGService(config=clarity_rag_config, qdrant_client=qdrant_client)
    
    start = time.time()
    rerank_results = rerank_service.retrieve(test_query, top_k=3)
    rerank_latency = time.time() - start
    
    print(f"⏱️  Latency: {rerank_latency*1000:.0f}ms")
    print(f"📄 Results: {len(rerank_results)}")
    for i, doc in enumerate(rerank_results, 1):
        print(f"   {i}. {doc.metadata.get('source', 'Unknown')[:50]}")
    
    print(f"\n📊 Comparison:")
    print(f"   Naive latency: {naive_latency*1000:.0f}ms")
    print(f"   Rerank latency: {rerank_latency*1000:.0f}ms")
    print(f"   Overhead: {(rerank_latency/naive_latency - 1)*100:.0f}%")
    
except Exception as e:
    print(f"⚠️  Rerank not available: {e}")
    print("   This is normal if Cohere API key is not set")

# Reset to naive
clarity_rag_config.retrieval_config.retriever_type = "naive"

print("\n✓ Strategy comparison complete")

## 7. Interactive Query Testing

Test your own custom queries.

In [ ]:
# Choose agent
agent_choice = "clarity"  # or "rigor"

# Your custom query
custom_query = "what makes mathematical writing clear and understandable"

print(f"\n🔍 Testing custom query for {agent_choice.upper()} agent")
print("="*60)
print(f"Query: {custom_query}\n")

# Select service
if agent_choice == "clarity":
    service = RAGService(config=clarity_rag_config, qdrant_client=qdrant_client)
else:
    service = RAGService(config=rigor_rag_config, qdrant_client=qdrant_client)

# Retrieve
start = time.time()
results = service.retrieve(custom_query, top_k=5)
latency = time.time() - start

print(f"⏱️  Latency: {latency*1000:.0f}ms")
print(f"📄 Retrieved {len(results)} documents\n")
print("="*60)

# Display formatted results
formatted = service.format_results(results)
print(formatted)

## 8. Diagnostic Information

In [ ]:
print("\n" + "="*60)
print("DIAGNOSTIC INFORMATION")
print("="*60)

# Check cache directories
print("\n📂 Cache Directories:")
cache_base = Path("app/embedding_cache")
for agent_type in ["clarity", "rigor", "test"]:
    cache_dir = cache_base / agent_type
    if cache_dir.exists():
        cache_files = list(cache_dir.glob("*"))
        total_size = sum(f.stat().st_size for f in cache_files if f.is_file())
        print(f"   {agent_type}: {len(cache_files)} files ({total_size/1024:.1f} KB)")
    else:
        print(f"   {agent_type}: Not found")

# Check Qdrant collections
print("\n📚 Qdrant Collections:")
try:
    collections = qdrant_client.get_collections()
    for collection in collections.collections:
        info = qdrant_client.get_collection(collection.name)
        print(f"   {collection.name}: {info.points_count} points")
except Exception as e:
    print(f"   Error: {e}")

# Configuration info
print("\n⚙️  Configuration:")
print(f"   Clarity chunk size: {clarity_rag_config.embedding_config.chunk_size}")
print(f"   Clarity top_k: {clarity_rag_config.retrieval_config.top_k}")
print(f"   Clarity retriever: {clarity_rag_config.retrieval_config.retriever_type}")
print(f"   Rigor chunk size: {rigor_rag_config.embedding_config.chunk_size}")
print(f"   Rigor top_k: {rigor_rag_config.retrieval_config.top_k}")
print(f"   Rigor retriever: {rigor_rag_config.retrieval_config.retriever_type}")

# Document resources
print("\n📄 Document Resources:")
for agent_type in ["clarity", "rigor"]:
    docs_dir = Path(f"app/resources/{agent_type}_docs")
    if docs_dir.exists():
        pdfs = list(docs_dir.glob("*.pdf"))
        print(f"   {agent_type}: {len(pdfs)} PDFs")
        for pdf in pdfs:
            print(f"      - {pdf.name}")
    else:
        print(f"   {agent_type}: Directory not found")

print("\n✓ Diagnostics complete")

## Summary

This notebook tested:

1. ✅ Qdrant connection and collections
2. ✅ Embedding cache functionality
3. ✅ Clarity agent RAG retrieval
4. ✅ Rigor agent RAG retrieval
5. ✅ RAG nodes workflow integration
6. ✅ Retrieval strategy comparison
7. ✅ Custom query testing
8. ✅ Diagnostic information

### Next Steps:

- If collections are empty: `python scripts/embed_documents.py --all`
- If cache is empty: Re-run embedding script
- If retrieval fails: Check Qdrant is running
- To add more documents: Add PDFs to `rag_resources/{agent}/` and re-embed

### Performance Targets:

- Cache speedup: >100x
- Naive retrieval: <200ms
- Rerank retrieval: <500ms
- Documents per agent: >10 chunks

Happy testing! 🚀